# 検索型質問応答

この例では、インデックス上での質問応答を紹介しています。

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA

In [ ]:
from langchain.document_loaders import TextLoader
loader = TextLoader("../../state_of_the_union.txt")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()
docsearch = Chroma.from_documents(texts, embeddings)

In [ ]:
qa = RetrievalQA.from_chain_type(llm=ChatOpenAI(model_name="gpt-4"), chain_type="stuff", retriever=docsearch.as_retriever())

In [ ]:
query = "What did the president say about Ketanji Brown Jackson"
qa.run(query)

## チェーンタイプ
RetrievalQAチェーンで使用する異なるチェーンタイプを簡単に指定できます。これらのタイプの詳細な手順については、[このノートブック](question_answering.ipynb)を参照してください。

異なるチェーンタイプをロードする方法は2つあります。まず、`from_chain_type`メソッドでチェーンタイプ引数を指定できます。これにより、使用するチェーンタイプの名前を渡すことができます。たとえば、以下の例では、チェーンタイプを`map_reduce`に変更しています。

In [ ]:
qa = RetrievalQA.from_chain_type(llm=ChatOpenAI(), chain_type="map_reduce", retriever=docsearch.as_retriever())

In [ ]:
query = "ケタンジ・ブラウン・ジャクソンについて、大統領はどのように語っていたのでしょうか。"
qa.run(query)

このようにして、本当に簡単にチェーンタイプを変更することができますが、そのチェーンタイプに対するパラメータに対して高い柔軟性を提供することができます。これらのパラメータを制御したい場合は、チェーンを直接ロードして（[このノートブック](question_answering.ipynb)で行ったように）、それを `combine_documents_chain` パラメータで RetrievalQAチェーンに直接渡すことができます。サンプルです：

In [ ]:
from langchain.chains.question_answering import load_qa_chain
qa_chain = load_qa_chain(ChatOpenAI(temperature=0), chain_type="stuff")
qa = RetrievalQA(combine_documents_chain=qa_chain, retriever=docsearch.as_retriever())

In [ ]:
query = "ケタンジ・ブラウン・ジャクソンについて、大統領はどのように語っていたのでしょうか。"
qa.run(query)

## カスタムプロンプト
質問応答を行うためにカスタムプロンプトを渡すことができます。これらのプロンプトは、[ベースの質問応答チェーン](./question_answering.ipynb)に渡すことができるプロンプトと同じです。

In [ ]:
from langchain.prompts import PromptTemplate
prompt_template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Answer in Japanese:"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

In [ ]:
chain_type_kwargs = {"prompt": PROMPT}
qa = RetrievalQA.from_chain_type(llm=ChatOpenAI(), chain_type="stuff", retriever=docsearch.as_retriever(), chain_type_kwargs=chain_type_kwargs)

In [ ]:
query = "What did the president say about Ketanji Brown Jackson"
qa.run(query)

## ソースドキュメントを返す
さらに、チェーンを構築する際にオプションパラメータを指定することで、質問に答えるために使用されたソースドキュメントを返すことができます。

In [ ]:
qa = RetrievalQA.from_chain_type(llm=ChatOpenAI(), chain_type="stuff", retriever=docsearch.as_retriever(), return_source_documents=True)

In [ ]:
query = "What did the president say about Ketanji Brown Jackson"
result = qa({"query": query})

In [ ]:
result["result"]

In [ ]:
result["source_documents"]